In [12]:
import json

import requests

 

#response = requests.get('https://ghoapi.azureedge.net/api/Indicator?$filter=contains(IndicatorName,%20%27Medical%20doctors%27)')
medical_doctors_per_10k = requests.get('https://ghoapi.azureedge.net/api/HWF_0001?$filter=date(TimeDimensionBegin) ge 2017-01-01 and date(TimeDimensionBegin) lt 2018-01-01').json()
#HWF_0001 is the indicator code for medical doctors per 10000, we then filter by date


country_list_code = requests.get('https://ghoapi.azureedge.net/api/DIMENSION/COUNTRY/DimensionValues').json()

In [62]:

import pandas as pd
from pandas import json_normalize

#flattened_data = json_normalize(country_list_code['value'])
country_df = pd.DataFrame(country_list_code['value'])

country_df.head(2)

,Code,Title,Dimension,ParentDimension,ParentCode,ParentTitle
0,AGO,Angola,COUNTRY,REGION,AFR,Africa
1,BDI,Burundi,COUNTRY,REGION,AFR,Africa


In [61]:
doctors_df = pd.DataFrame(medical_doctors_per_10k['value'])[['SpatialDim','NumericValue']]
doctors_df.head(2)

,SpatialDim,NumericValue
0,AGO,2.146
1,ARE,24.356


In [41]:
malaria_estimated_numbers = pd.read_csv('./estimated_numbers.csv')

In [86]:
malaria_estimated_numbers_2017 = malaria_estimated_numbers[malaria_estimated_numbers['Year'].between(2017, 2017)]
malaria_estimated_numbers_2017.head(2)#.sort_values(by=['No. of deaths'])

,Country,Year,No. of cases,No. of deaths,No. of cases_median,No. of cases_min,No. of cases_max,No. of deaths_median,No. of deaths_min,No. of deaths_max,WHO Region
0,Afghanistan,2017,630308[495000-801000],298[110-510],630308,495000.0,801000.0,298,110.0,510.0,Eastern Mediterranean
1,Algeria,2017,0,0,0,NaN,NaN,0,NaN,NaN,Africa


In [96]:
merged_gho_df = country_df.merge(doctors_df, left_on='Code', right_on='SpatialDim')[['Title','ParentTitle','NumericValue']]
merged_gho_df.columns = ['Country', 'Continent','Doctors/10k']
merged_gho_df.head(2)

,Country,Continent,Doctors/10k
0,Angola,Africa,2.146
1,Burundi,Africa,1.001


In [98]:
gho_plus_malaria_df = merged_gho_df.merge(malaria_estimated_numbers_2017, left_on='Country', right_on='Country')[['Country','Continent','Doctors/10k','Year','No. of cases_median','No. of deaths_median']]
gho_plus_malaria_df.columns = ['Country','Continent','Doctors/10k','Year','No of sick People', 'No of dead people']

In [103]:
gho_plus_malaria_df.sort_values(by=['No of dead people'], ascending=False).head(10)


,Country,Continent,Doctors/10k,Year,No of sick People,No of dead people
2,Burkina Faso,Africa,0.847,2017,7907562,27791
46,India,South-East Asia,7.779,2017,9590000,16733
11,Mozambique,Africa,0.761,2017,10025823,14713
6,Uganda,Africa,1.680,2017,8600724,14390
0,Angola,Africa,2.146,2017,4615605,13316
16,Ghana,Africa,0.920,2017,7805045,10917
4,Chad,Africa,0.434,2017,2779489,8729
5,Togo,Africa,0.757,2017,2891868,5341
1,Burundi,Africa,1.001,2017,2113066,5253
8,Senegal,Africa,0.691,2017,1024285,4523
